In [153]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from google_auth_oauthlib import flow
from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.oauth2 import service_account
import requests
import json
import csv

SERVICE_ACCOUNT_FILE = 'key.json'

credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE)

bqclient = bigquery.Client(project='iotube-analytics', credentials=credentials)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [217]:
COVALENT_KEY = "ckey_10446544a51944ffa7c6603a9e4"
chain = {
    "Polygon": 137,
    "Ethereum": 1,
    "BSC": 56
}
validator = {
    "Polygon": "0xFBe9A4138AFDF1fA639a8c2818a0C4513fc4CE4B",
    "Ethereum": "0xd8165188ccc135b3a3b2a5d2bc3af9d94753d955",
    "BSC": "0x116404F86e97846110EA08cd52fC2882d4AD3123"
}

In [218]:
def getBridgeDataIn(network):
    '''
    Gets inflow bridge data for a given network
    '''
    dfToIotex = pd.DataFrame(columns = ['Network', 'Date', 'Address', 'Transaction Fee', 'Token Name', 'Token Symbol', 'Value'])
    endpoint = "https://api.covalenthq.com/v1/{}/address/{}/transactions_v2/?page-size={}&key={}".format(chain[network], validator[network], 10000, COVALENT_KEY)
    response = requests.get(endpoint)
    JSON_DATA_RAW = response.json()
    JSON_DATA_RAW = JSON_DATA_RAW['data']

    for item in JSON_DATA_RAW['items']:
        temp = []
        temp.append(network)
        temp.append(item['block_signed_at'].split("T", 1)[0]) # Date
        temp.append(item['from_address']) # Address 
        temp.append(item['gas_price'] * item['gas_spent'] * pow(10, -18)) # Transaction Fee
        for event in item['log_events']:
            if(event['decoded']):
                if(event['decoded']['name'] == "Transfer"):
                    for param in event['decoded']['params']:
                        if(param["name"] == "to"):
                            address = param['value'] 
                        if(param["name"] == "value"):
                            value = param['value']
                    name = event['sender_name']
                    symbol = event['sender_contract_ticker_symbol']
                    decimals = -event['sender_contract_decimals']
                    value = int(value) * pow(10,decimals)
        temp.append(name) # Ticker Name
        temp.append(symbol) # Ticker Symbol
        temp.append(value) # Ticker Value
        dfToIotex.loc[len(dfToIotex)] = temp
                            
    dfToIotex['Date'] = pd.to_datetime(dfToIotex['Date'])
    return dfToIotex

In [225]:
polygon = getBridgeDataIn("Polygon")
ethereum = getBridgeDataIn("Ethereum")
bsc = getBridgeDataIn("BSC")
dfToIotex = polygon.append([ethereum, bsc])

In [247]:
inflow = pd.read_csv('https://storage.googleapis.com/iotube/bridgeInflowRaw')
inflow.head()

,Unnamed: 0,Network,Date,Address,Transaction Fee,Token Name,Token Symbol,Value
0,0,Polygon,2021-07-01,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,1.0
1,1,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000032,Wrapped Matic,WMATIC,1.0
2,2,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000032,Wrapped Matic,WMATIC,1.0
3,3,Polygon,2021-06-30,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,5.0
4,4,Polygon,2021-06-17,0x6dd31a526ee3ddbc7be888b729a445695c03148e,0.000121,Wrapped Matic,WMATIC,1.0


In [263]:
def crossChainGrouping(inDf):
    inflowDf = inDf.groupby(["Network", "Date"]).agg({'Value': ['count', 'sum', 'mean', 'median'],
                                                                'Transaction Fee': ['sum', 'mean', 'median']})
    inflowDf['Value']['sum'] = inflowDf['Value']['sum'].round()
    inflowDf['Transaction Fee']['sum'] = inflowDf['Transaction Fee']['sum']

    inflowDf = inflowDf.reset_index()
    inflowDf.columns = inflowDf.columns.droplevel(0)
    inflowDf.columns = ['Network', 'Date', 'Frequency', 'Volume', 'Mean Volume', 'Median Volume', 'Transaction Fee', 'Mean Txn Fee', 'Median Txn Fee']
    
    return inflowDf

def tokenGrouping(inDf):
    '''
    Bridge data grouping by token
    '''
    dfInGroupedToken = inDf.groupby(["Token Symbol", "Date"], as_index=False).agg({'Value': ['count', 'sum', 'mean', 'median']})
    dfInGroupedToken['Value']['sum'] = dfInGroupedToken['Value']['sum'].round()

    dfInGroupedToken = dfInGroupedToken.reset_index()
    dfInGroupedToken.columns = dfInGroupedToken.columns.droplevel(0)
    dfInGroupedToken.columns = ['', 'Token Symbol', 'Date', 'Frequency', 'Volume', 'Average Value', 'Median']

    dfInGroupedPeriod = inDf.groupby(["Network", "Token Symbol", "Date"], as_index=False).agg({'Value': ['count', 'sum', 'mean', 'median']})
    dfInGroupedPeriod['Value']['sum'] = dfInGroupedPeriod['Value']['sum'].round()

    dfInGroupedPeriod = dfInGroupedPeriod.reset_index()
    dfInGroupedPeriod.columns = dfInGroupedPeriod.columns.droplevel(0)
    dfInGroupedPeriod.columns = ['', 'Network', 'Token Symbol', 'Date', 'Frequency', 'Volume', 'Average Value', 'Median']
    
    return dfInGroupedToken, dfInGroupedPeriod

In [264]:
dfInGroupedToken, dfInGroupedPeriod = tokenGrouping(inflow)
chained = crossChainGrouping(inflow)

<ipython-input-263-887224c81bbe>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfInGroupedToken['Value']['sum'] = dfInGroupedToken['Value']['sum'].round()
<ipython-input-263-887224c81bbe>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfInGroupedPeriod['Value']['sum'] = dfInGroupedPeriod['Value']['sum'].round()
<ipython-input-263-887224c81bbe>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [265]:
chained

,Network,Date,Frequency,Volume,Mean Volume,Median Volume,Transaction Fee,Mean Txn Fee,Median Txn Fee
0,BSC,2021-02-25,4,0.040000,0.010000,0.010,0.002448,0.000612,0.000537
1,BSC,2021-03-11,2,1.010000,0.505000,0.505,0.002252,0.001126,0.001126
2,BSC,2021-03-12,6,3.130000,0.521667,0.055,0.006973,0.001162,0.001084
3,BSC,2021-03-13,3,0.050000,0.016667,0.010,0.003884,0.001295,0.001325
4,BSC,2021-03-14,6,5.454576,0.909096,0.020,0.007647,0.001274,0.001324
...,...,...,...,...,...,...,...,...,...
231,Polygon,2021-06-14,2,2.500000,1.250000,1.250,0.000170,0.000085,0.000085
232,Polygon,2021-06-15,3,4.100000,1.366667,2.000,0.000291,0.000097,0.000121
233,Polygon,2021-06-17,5,8.000000,1.600000,2.000,0.000604,0.000121,0.000121
234,Polygon,2021-06-30,3,7.000000,2.333333,1.000,0.000185,0.000062,0.000032
